#### 데이터 로드

In [2]:
import pandas as pd

seoul_starbucks = pd.read_excel('crawling_data/4_1_seoul_starbucks_list.xlsx', header = 0)
seoul_starbucks.head()

매장명         위도          경도     매장타입                          주소  \
0  역삼아레나빌딩  37.501087  127.043069  general     서울특별시 강남구 언주로 425 (역삼동)   
1   논현역사거리  37.510178  127.022223  general    서울특별시 강남구 강남대로 538 (논현동)   
2  신사역성일빌딩  37.514132  127.020563  general    서울특별시 강남구 강남대로 584 (논현동)   
3   국기원사거리  37.499517  127.031495  general    서울특별시 강남구 테헤란로 125 (역삼동)   
4   스탈릿대치R  37.494668  127.062583  reserve  서울특별시 강남구 남부순환로 2947 (대치동)   

        전화번호  
0  1522-3232  
1  1522-3232  
2  1522-3232  
3  1522-3232  
4  1522-3232

In [22]:
# 스타벅스 주소에서 시군구명 정보 추출
sgg_names = []
for address in seoul_starbucks['주소']:
    sgg = address.split(' ')[1]
    sgg_names.append(sgg)

seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks = seoul_starbucks[seoul_starbucks['시군구명'] != '67']
seoul_starbucks.head()

매장명         위도          경도     매장타입                          주소  \
0  역삼아레나빌딩  37.501087  127.043069  general     서울특별시 강남구 언주로 425 (역삼동)   
1   논현역사거리  37.510178  127.022223  general    서울특별시 강남구 강남대로 538 (논현동)   
2  신사역성일빌딩  37.514132  127.020563  general    서울특별시 강남구 강남대로 584 (논현동)   
3   국기원사거리  37.499517  127.031495  general    서울특별시 강남구 테헤란로 125 (역삼동)   
4   스탈릿대치R  37.494668  127.062583  reserve  서울특별시 강남구 남부순환로 2947 (대치동)   

        전화번호 시군구명  
0  1522-3232  강남구  
1  1522-3232  강남구  
2  1522-3232  강남구  
3  1522-3232  강남구  
4  1522-3232  강남구

In [23]:
# 엑셀로 저장
seoul_starbucks.to_excel('crawling_data/4_4_seoul_starbucks_list.xlsx', index = False)

In [24]:
# 시군구 목록 데이터 불러오기
seoul_sgg = pd.read_excel('crawling_data/4_2_seoul_sgg_list.xlsx')
seoul_sgg.head()

시군구코드  시군구명         위도          경도
0  11320   도봉구  37.665861  127.031767
1  11380   은평구  37.617612  126.922700
2  11230  동대문구  37.583801  127.050700
3  11590   동작구  37.496504  126.944307
4  11545   금천구  37.460097  126.900155

In [31]:
# 시군구별 스타벅스 매장 수 세기
starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명', values = '매장명', aggfunc = 'count').rename(columns = {'매장명' : '스타벅스_매장수'})
starbucks_sgg_count.head()

스타벅스_매장수
시군구명          
강남구         80
강동구         14
강북구          5
강서구         16
관악구         11

In [35]:
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on = '시군구명')
seoul_sgg.head()

시군구코드  시군구명         위도          경도  스타벅스_매장수
0  11320   도봉구  37.665861  127.031767         2
1  11380   은평구  37.617612  126.922700         8
2  11230  동대문구  37.583801  127.050700         8
3  11590   동작구  37.496504  126.944307        11
4  11545   금천구  37.460097  126.900155        10

In [36]:
# 인구통계 데이터 불러오기
seoul_sgg_pop = pd.read_excel('crawling_data/4_3_sgg_pop.xlsx')
seoul_sgg_pop.head()

시군구명   주민등록인구
0  종로구  177,023
1   중구  138,102
2  용산구  250,844
3  성동구  316,113
4  광진구  386,292

In [38]:
# 스타벅스 데이터와 인구통게 데이터 병합
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how='left', on='시군구명')
seoul_sgg.head()

시군구코드  시군구명         위도          경도  스타벅스_매장수   주민등록인구
0  11320   도봉구  37.665861  127.031767         2  371,095
1  11380   은평구  37.617612  126.922700         8  474,165
2  11230  동대문구  37.583801  127.050700         8  374,039
3  11590   동작구  37.496504  126.944307        11  407,966
4  11545   금천구  37.460097  126.900155        10  262,337

In [39]:
# 서울시 시군구별 사업체 수 통계 데이터도 병합
seoul_sgg_biz = pd.read_excel('crawling_data/4_3_sgg_biz.xlsx')
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_biz, how = 'left', on = '시군구명')
seoul_sgg.head()

시군구코드  시군구명         위도          경도  스타벅스_매장수   주민등록인구    종사자수   사업체수
0  11320   도봉구  37.665861  127.031767         2  371,095   68669  18455
1  11380   은평구  37.617612  126.922700         8  474,165   87693  24179
2  11230  동대문구  37.583801  127.050700         8  374,039  143858  32994
3  11590   동작구  37.496504  126.944307        11  407,966  103915  19609
4  11545   금천구  37.460097  126.900155        10  262,337  223058  30080

In [40]:
# 전처리 된 결과를 엑셀 파일로 저장
seoul_sgg.to_excel('crawling_data/4_5_seoul_sgg_stat.xlsx', index = False)